In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = "/kaggle/input/elgpt-advance/final_t5-base_medical_chatbot"  # Path where you saved the model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)



2025-05-17 22:55:42.677452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747522542.701218     116 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747522542.708704     116 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:

def generate_response(patient_query):
    input_text = f"### Instruction:\n{patient_query}"
    
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,
        repetition_penalty=1.2,
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [3]:
test_queries = [
    "I have a severe headache and sensitivity to light. What could this be?",
    "What are the symptoms of diabetes?",
    "How can I treat a minor burn at home?",
    "Hello doctor, I am six weeks pregnant and suffering from severe constipation which is the main cause of heartburn and acidity. Can I take any stool softener?"
]

print("Testing model responses:")
for query in test_queries:
    print("\nPatient:", query)
    print("Doctor:", generate_response(query))
    print("-" * 50)

Testing model responses:

Patient: I have a severe headache and sensitivity to light. What could this be?
Doctor: HiThanks for using healthcare magicI think you have severe headache due to light exposure. If it is mildly irritating, then please consult a psychiatrist online who can help you find out more information about migraine treatment options available here. Hope this helps! Regarding Headache: For further queries consult an internal medicine physician online -->
--------------------------------------------------

Patient: What are the symptoms of diabetes?
Doctor: Hi,I have gone through your query with diligence and would like you to know that I am here to help you.Diabetes are often associated with a lack of appetite or low blood sugar levels due to diabetes type 2 (Table 3). Diabetic Type 1 is usually diagnosed by the presence of insulin in urine which can lead to hypertension. In addition, it may be related to other types of Diabetes such as adrenal gland disease, kidney ston

In [4]:
pip install torch transformers langchain gradio sacremoses

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.prompts import PromptTemplate

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import gradio as gr

In [7]:
pip install -U langchain langchain-core langchain-community langchain-huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_huggingface import HuggingFacePipeline

In [9]:
from langchain_core.prompts.prompt import PromptTemplate
from transformers import pipeline

pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=False,
    repetition_penalty=1.2,
)

template = """
Below is a conversation between a Patient and a Doctor.

Previous Conversation:
{history}

Current Question:
Patient: {input}
Doctor:"""

Device set to use cuda:0


In [ ]:
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

memory = ConversationBufferMemory(
    return_messages=True,
    ai_prefix="Doctor's Response",
    human_prefix="Current Question",
    memory_key="history"
)

llm = HuggingFacePipeline(pipeline=pipeline)

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory,
    prompt=prompt
)

/tmp/ipykernel_116/3888951865.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/tmp/ipykernel_116/3888951865.py:18: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [ ]:
class ChatbotWithMemory:
    def __init__(self):
        self.conversation_history = []
        
    def predict(self, input_text):
        response = conversation.predict(input=input_text)
        self.conversation_history.append((input_text, response))
        return self.conversation_history
    
    def clear_history(self):
        self.conversation_history = []
        conversation.memory.clear()
        return []

chatbot = ChatbotWithMemory()

In [ ]:
with gr.Blocks() as demo:
    chat_interface = gr.Chatbot(height=400)
    msg = gr.Textbox(label="Type your medical question")
    clear = gr.Button("Clear Conversation")
    
    def user(user_message, history):
        updated_history = chatbot.predict(user_message)
        return "", updated_history
    
    def clear_chat():
        return chatbot.clear_history()
    
    msg.submit(user, [msg, chat_interface], [msg, chat_interface])
    clear.click(clear_chat, None, chat_interface)

demo.launch(share=True)

/tmp/ipykernel_116/277802523.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_interface = gr.Chatbot(height=400)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://09211235b27563f06f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new ConversationChain chain...
Prompt after formatting:

Below is a conversation between a Patient and a Doctor.

Previous Conversation:
[HumanMessage(content='My knees are experiencing alot of pain', additional_kwargs={}, response_metadata={}), AIMessage(content='DearWe understand your concernsI went through the details. I suggest you not to worry much about knee pains due to arthritis or other inflammation related problems like muscle cramping, swelling and arthropathy. If there is any problem with knee pain then please consult an orthopaedician online for further information.', additional_kwargs={}, response_metadata={})]

Current Question:
Patient: My knees are experiencing alot of pain
Doctor:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:

Below is a conversation between a Patient and a Doctor.

Previous Conversation:
[HumanMessage(content='My knees are experiencing alot of pain', additional_kwargs={}, response_metadata={}), A